In [1]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

parent_dir = os.path.abspath(os.path.join(os.getcwd().split('render')[0],'gaussian_data'))
sys.path.insert(0, parent_dir)

from gaussian_data.Camera import Camera
from gaussian_data.Frame import Frame
import gaussian_data.Plotters
import gaussian_data.Utils as Utils

from GaussianSplat import GaussianSplat


import numpy as np
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import Plotters
import plotly
import plotly.graph_objects as go
import scipy.io
import pandas as pd
from Render import Render

%matplotlib qt

# load hull
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
# initilize objects
frames = [1408]#list(range(500,700,13))

image_names,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_names)}
# map 3d voxels to 2d pixels
[frames[im_name].map_3d_2d(croped_image = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)


In [2]:
# load gaussian splatting output
input_file = "G:/My Drive/Research/gs_data/mov19_2022_03_03/output/frame140845_scaling_lr0.005_percent_dense0.0001_densify_grad_threshold0.0002_position_lr_init12/point_cloud/iteration_60000/point_cloud.ply"
# input_file = "D:\Documents/2dgs_output/point_cloud3.ply"
input_file = "G:/My Drive/Research/gs_data/mov19_2022_03_03/time/time2_norm01_dist1000000_iter200_start100_moreFr/1408/point_cloud/iteration_200/point_cloud.ply"
# input_file = "G:/My Drive/Research/gs_data/mov19_2022_03_03/2dgs_output/93701c9b-c2/point_cloud/iteration_1000/point_cloud.ply"

gs = GaussianSplat(input_file)  
gs.save_gs('_2dsplat')
# reproject and filter background
gs_filtered = gs.filter(gs.projection_filter(frames,gs.xyz,croped_image = True),path = gs.path)
gs_filtered.save_gs('_filtered')




In [7]:
cam = frames[f'{image_names[2]}.jpg'].get_croped_camera()
render = Render(gs,cam,tiles = [1,10],block_xy = [16,16], image_size = [160,160],gaus3d = False,filtersze = np.sqrt(2) / 2)

image = render.render_image()


In [8]:
plt.figure(),plt.imshow(image)

(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x2982aebdc00>)

In [4]:
# render image 
image_name = f'{image_names[0]}.jpg'

cam1 = frames[image_name].get_croped_camera()
cam4 = frames[f'{image_names[0]}.jpg'].get_croped_camera()

rotmat = np.array([[1,0,0],[0,1,0],[0,0,1]])

K_crop = cam4.K
K_crop[0,0] = K_crop[0,0]
K_crop[1,1] = K_crop[1,1]

R = rotmat @ cam4.R
X0 = cam4.X0 + np.array([[0,0,0]]).T
cam = Camera(cam1.path,5,cam = {'camera': np.hstack((K_crop,R,X0))[:, :, np.newaxis]},image_size = [160,160])

cam = frames[f'{image_names[0]}.jpg'].get_croped_camera()



render = Render(gs,cam,tiles = [1,10],block_xy = [16,16], image_size = [160,160])
image = render.render_image()
plt.figure(),plt.imshow(image)
# plt.figure(),plt.imshow(render.depth)
np.unique(render.depth)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


array([0.        , 0.00112268, 0.00112602, ..., 0.28472146, 0.28472418,
       1.        ])

In [8]:
cam.K_crop

AttributeError: 'Camera' object has no attribute 'K_crop'

In [5]:
plt.imshow(np.mean(image,2)*255)

In [29]:
frames.keys()

dict_keys(['P1408CAM1.jpg', 'P1408CAM2.jpg', 'P1408CAM3.jpg', 'P1408CAM4.jpg'])

In [32]:
plt.imshow(np.array(frames['P1408CAM3.jpg'].croped_image) - np.mean(image,2)*255)

In [6]:
import sh_utils

vertices = PlyData.read(input_file)["vertex"]


sh = np.column_stack([vertices[key] for key in vertices.data.dtype.names if 'rest' in key or 'dc' in key])
rgb = sh_utils.rgb_from_sh(0,sh,xyz = None,camera_position = None)
xyz = np.column_stack((vertices["x"], vertices["y"], vertices["z"]))

[frames[im_name].map_3d_2d(croped_image = True, use_zbuff = True) for im_name in frames.keys()]
fig = go.Figure()
color_pts = ['red','green','blue','black']
fig = go.Figure()
im_name = list(frames.keys())[0]
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    Plotters.scatter3d(fig,frames[image].voxels_with_idx,color_pts[cam],3)
opacity = 1 / (1 + np.exp(-vertices["opacity"]))   

vertices = vertices[(rgb[:,0] < 0.9) & (rgb[:,0] > 0) & (opacity > 0.1) & (rgb[:,2] < 0.9) & (rgb[:,2] > 0) & (rgb[:,1] < 0.9) & (rgb[:,1] > 0)]
rgb = rgb[(rgb[:,0] < 0.9) & (rgb[:,0] > 0) & (opacity > 0.1) & (rgb[:,2] < 0.9) & (rgb[:,2] > 0) & (rgb[:,1] < 0.9) & (rgb[:,1] > 0)]

# Create the 3D scatter plot with hover template for color value
fig.add_trace(go.Scatter3d(
    x=vertices["x"],
    y=vertices["y"],
    z=vertices["z"],
    mode='markers',
    marker=dict(
        size=3,
        color=rgb[:,0],  # Color for each point
        colorscale='Gray',  # Choose a colorscale
        opacity=0.5,  # Use the opacity per point
        colorbar=dict(title="Colorbar")  # Optional colorbar
    ),
    hovertemplate='Color: %{marker.color:.4f}<extra></extra>',  # Show color as float
))
fig.show()


In [5]:
def build_rotation(r):
    norm = np.sqrt(r[:,0]*r[:,0] + r[:,1]*r[:,1] + r[:,2]*r[:,2] + r[:,3]*r[:,3])

    q = r / norm[:, None]

    R = np.zeros((q.shape[0], 3, 3))

    r = q[:, 0]
    x = q[:, 1]
    y = q[:, 2]
    z = q[:, 3]

    R[:, 0, 0] = 1 - 2 * (y*y + z*z)
    R[:, 0, 1] = 2 * (x*y - r*z)
    R[:, 0, 2] = 2 * (x*z + r*y)
    R[:, 1, 0] = 2 * (x*y + r*z)
    R[:, 1, 1] = 1 - 2 * (x*x + z*z)
    R[:, 1, 2] = 2 * (y*z - r*x)
    R[:, 2, 0] = 2 * (x*z - r*y)
    R[:, 2, 1] = 2 * (y*z + r*x)
    R[:, 2, 2] = 1 - 2 * (x*x + y*y)
    return R
def build_scaling_rotation(s, r):
    L = np.zeros((s.shape[0], 3, 3))
    R = build_rotation(r)

    L[:,0,0] = s[:,0]
    L[:,1,1] = s[:,1]
    L[:,2,2] = s[:,2]

    L = R @ L
    return L

def intersection_point(pixel,T):
    k = -T[..., 0] + pixel[0]*T[...,3]
    l = -T[..., 1] + pixel[1] * T[..., 3]
    points = np.cross(k, l, axis=-1)
    return points[..., :2] / points[..., -1:]

def homogeneous(points):
    """
    homogeneous points
    :param points: [..., 3]
    """
    return np.column_stack((points, np.ones(points.shape[0])))

def homogeneous_vec(vec, vectoadd = [0,0]):
    """
    homogeneous points
    :param points: [..., 3]
    """
    return np.concatenate((vec,np.tile(np.array([vectoadd]).T,(vec.shape[0],1,1))),axis = 2)


# Surface splatting (2D Gaussian Splatting)
def setup(means3D, scales, quats, opacities, colors, viewmat, projmat):
    rotations = build_scaling_rotation(scales, quats)

   
    # 1. Viewing transform
    # Eq.4 and Eq.5
    p_view = (means3D @ viewmat[:3,:3]) + viewmat[-1:,:3] # rotate the gaussian mean to camera FoR
    uv_view = (rotations @ viewmat[:3,:3]) # rotate to camera FoR

    # M is H matrix that representes the transformation from tangent plane to camera. 
    # its the scaled axes concatenated to the gaussian mean location - represented in homogeneous coordinates

    # !! need to check that the order of axes ar ok for M !!
    M = np.concatenate((homogeneous_vec(uv_view[:,:2,:]),homogeneous(p_view)[:,np.newaxis]),axis = 1)
    T = M @ projmat # T stands for (WH)^T in Eq.9 - projmat transforms from camera to NDC (screen coordinates)
    # T is the transformation of every gaussian from tangent plane to NDC, its homogebnus coordinates. with the rotation matrix 
    # representing the axes and the translation vector representing the location of the center of each gaussian. 
    # We notice that projmat is a prespective projection matrix. 

    # Next, We calculate the radius of the gaussian. We normalize by w to get homogeneus coordinates. In addition we flip Z axis (not sure why) 
    # we calculate the distance from the camera to the gaussian mean (this is w, the last row of a homogenues coordinate, deviding by it will give perspective view)
    # Notice that the rotation is scaled (in build_scaling_rotation) and is not normalized.

    # point_image - the projectes mean of the gaussian (with flipped z)
    # half_extend - used to calculate the radius of the gaussian, we take 3 sigma. because the ratation is scaled 
    # we calculate the distance for each axis and can get the 3 sigma by multiplying each distance. (we also devide by w to get the prespective view)

    temp_point = np.tile([1,1,-1],(T.shape[0],1))
    distance  = np.sum(temp_point*T[..., 3] * T[..., 3],-1)
    f = (1 / distance[:,np.newaxis]) * temp_point

    # distance = (temp_point * (T[..., 3] * T[..., 3])).sum(dim=-1, keepdims=True)
    point_image = np.column_stack((np.sum(f * T[..., 0] * T[...,3],1),np.sum(f * T[..., 1] * T[...,3],1),np.sum(f * T[..., 2] * T[...,3],1)))

    axes_dist = np.column_stack((np.sum(f * T[..., 0] * T[...,0],1),np.sum(f * T[..., 1] * T[...,1],1),np.sum(f * T[..., 2] * T[...,2],1)))

    half_extend = point_image * point_image - axes_dist
    radii = np.sqrt(np.maximum(half_extend, 1e-4)) * 3
    center = point_image

    # 3. Perform Sorting
    depth = p_view[..., 2] # depth is used only for sorting
    index = np.argsort(depth)
    T = T[index]
    colors = colors[index]
    center = center[index]
    depth = depth[index]
    radii = radii[index]
    return T, colors, opacities, center, depth, radii



intrins = frames['P1408CAM1.jpg'].K_crop
projmat = np.zeros((4,4))
projmat[:3,:3] = intrins
projmat[-1,-2] = 1.0
projmat = projmat.T
viewmat = frames['P1408CAM1.jpg'].world_to_cam.T
T, colors, opacities, center, depth, radii = setup(gs.xyz, gs.scale, gs.rot, gs.opacity, gs.color, viewmat, projmat)

# # Rasterization
# # 1. Generate pixels
# W, H = 80,160#int(intrins[0, -1] * 2), int(intrins[1, -1] * 2)
# pix_x, pix_y = np.meshgrid(np.arange(W), np.arange(H), indexing='xy')
# pix = np.stack([pix_x, pix_y], axis=-1)

# # 2. Compute ray splat intersection # Eq.9 and Eq.10
# pix_flat = pix.reshape(-1, 1, 2)
# pixels = np.squeeze(pix_flat)
# x,y = pixels[:,0],pixels[:,1]
# s = np.vstack([intersection_point(pixel,T) for pixel in pixels])


# # 3. Add 
# # -pass filter # Eq.11
# # When a point (2D Gaussian) is viewed from a far distance or at a slanted angle,
# # the 2D Gaussian falls between pixels, and no fragment is used to rasterize the Gaussian.
# # Add a low-pass filter to handle aliasing.
# dist3d = np.sum(s * s, axis=-1)
# dist3d = np.reshape(dist3d,(len(pixels),T.shape[0]))

# filtersze = np.sqrt(2) / 2
# dist_xycenter = np.hstack([pixel - center[None, :, :2] for pixel in pixels])
# dist2d = (1 / filtersze) ** 2 * np.linalg.norm(dist_xycenter, axis=-1) ** 2
# dist2d = np.reshape(dist2d,(len(pixels),T.shape[0]))
# # Min of dist2 is equal to max of Gaussian exp(-0.5 * dist2)
# dist2 = np.minimum(dist3d, dist2d)

# depth_acc = np.sum(homogeneous(s)*np.tile(T[..., -1],(len(pixels),1)),axis = 1)
# depth_acc = np.reshape(depth_acc,(len(pixels),T.shape[0]))

In [7]:
gs.scale

array([[8.06498141e-05, 7.69689944e-05, 1.60000000e-06],
       [7.98608723e-05, 8.76289931e-05, 1.60000000e-06],
       [8.31628697e-05, 9.40807091e-05, 1.60000000e-06],
       ...,
       [9.92269857e-05, 8.62566106e-05, 1.60000000e-06],
       [8.34508671e-05, 5.89549979e-05, 1.60000000e-06],
       [7.89204542e-05, 8.14726523e-05, 1.60000000e-06]])

In [7]:
cam = frames[f'{image_names[0]}.jpg'].get_croped_camera()
render = Render(gs,cam,tiles = [1,10],block_xy = [16,16], image_size = [160,160],T = T,center = center,gaus3d = True)

image = render.render_image()
plt.figure(),plt.imshow(image)
# plt.figure(),plt.imshow(render.depth)
np.unique(render.depth)


NameError: name 'T' is not defined

In [71]:
# projection
xyz = np.column_stack((vertices["x"], vertices["y"], vertices["z"]))

im_name= 'P1408CAM1.jpg'
fig,axs = plt.subplots(2,2)
for cam in range(4):
    cam_obj  = frames[f'{image_names[cam]}.jpg'].get_croped_camera()

    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    homo_voxels_with_idx = frames[image].add_homo_coords(xyz)
    proj = cam_obj.project_with_proj_mat(xyz)
    axs[cam // 2,cam % 2].imshow(frames[image].croped_image)
    axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 3,c = rgb)

## Elliptical Gaussian (from EWA article) - Math explained


we define an elliptical gaussian: 

Equation 1: $G_\mathbf{V}(\mathbf{x} - \mathbf{p}) = \frac{1}{2 \pi |\mathbf{V}|} \frac{1}{2} e^{-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})}.$

* $V$ is the covariance matrix
* $\mathbf{x}$ - a 3d coordinate (in gaussian axes)
* $\mathbf{p}$ - the mean of the gaussian 
### Affine projection of the gauusian
We define an arbitrary affine mapping from object space $ \mathbf{u} = \Phi(\mathbf{x}) $, where $ \Phi(\mathbf{x}) = \mathbf{Mx + c} $, 

and we substitute $ \mathbf{x} = \Phi^{-1}(\mathbf{u}) $. Since $ \Phi^{-1}(\mathbf{u}) = \mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) $, we get:

$$
G_\mathbf{V}(\Phi^{-1}(\mathbf{u}) - \mathbf{p}) = 
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \mathbf{c} - \mathbf{Mp})^T \mathbf{M}^{-1} \mathbf{V}^{-1} (\mathbf{M}^{-1})^T (\mathbf{u} - \mathbf{c} - \mathbf{Mp})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \Phi(\mathbf{p}))^T (\mathbf{MVM}^T)^{-1} (\mathbf{u} - \Phi(\mathbf{p}))}.
$$

from this, to ease the writing we define equation 2: 

Equation 2: $G \mathbf{V}(\Phi^{-1}(u) - \mathbf{p}) = \frac{1}{|\mathbf{M}^{-1}|} G_{ \mathbf{M} \mathbf{V} \mathbf{M}^T} (u - \Phi(\mathbf{p})).$

#### The covariance matrix
$V$, the covariance matrix is a symetrix matrix that can be ragarded as a transformation. it scales and shears the data (thus defining the orientation and ratio between the gaussian principal axes). 
As mentioned above, the gaussian is transformed using an affine mapping $\Phi(X)$. Usually we use this kind of mapping to transform a vector or a 3D point to a new FoR (change basis). Because the covariance matrix is a transformation, in order to change its FoR (basis), we will perform $\mathbf{V_{1}} = MVM^{T}$. We first multiply $VM^{T}$  - transform (whatever xyz we multiply) from camera to body ($M^{T}$) and then multiply by $V$ which is in object coordinates - now we have coordinates in object basis after shear and scale. we then multiply by $M$ again to transform back to camera coordinates. 


### Projection


![alt text](ray_ewa.png)
#### Ray space

We converted to camera space, now we want to convert to ray space. The coordinates in ray space are defined such that $x_0, x_1$ are the coordinates on the image plane and the devision by $u_2$ adds prespective. $u_3$ is then defined as the euclidean distance from the point to the origin (camera). 


$$ \text{ray space to camera: }
\begin{pmatrix}
x_0 \\
x_1 \\
x_2 \\

\end{pmatrix} = m(u) = \begin{pmatrix}
u_0 / u_2 \\
u_1/u_2 \\
||(u_0, u_1, u_2)||^{T} \\
\end{pmatrix}

\text{,camera to ray space: }
\begin{pmatrix}
u_0 \\
u_1 \\
u_2 \\

\end{pmatrix} = m^{-1}(x) = \begin{pmatrix}
x_0/||(x_0,x_1,1)^T|| \cdot x_2 \\
x_1/||(x_0,x_1,1)^T|| \cdot x_2 \\
1/||(x_0,x_1,1)^T|| \cdot x_2 \\

\end{pmatrix}


$$
  
Ray space is defined with 3 coordinates $(x_0,x_1,x_2)$ where $x_0$ and $x_1$ represents the location of the pixel and $x_3$ is the euclidean distance from the camera origin to $(x_1,x_2). This transformation is not affine due to the devision by $u_2$, parallel lines are not preserved and the shape of the gaussian will be distorted. (in addition, the dimention is problematic).
In order to solve this a local affine approximation $\mathbf{m_{u_k}}$ of the projection transformation is defined. The approximation is the first to terms of the Tayor expansion of $\mathbf{m}$ at point $\mathbf{u}_{k}$:
$$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
where $\mathbf x_k=\mathbf m(\mathbf u_k)$ is the mean of the gaussian in ray space (can be transformed using the mapping m, the projection matrix) and the Jacobian $ J_{u_k} = \frac{\partial m}{\partial u}(u_k) $

### The gaussian mapping
#### Object to camera coordinates (as detailed in Affine projection of the gauusian)

The reconstruction kernels are initially given in object space, which has coordinates $t =(t_0,t_1,t_2)^T$ . 
* Gaussian reconstruction kernels in object space: $r"_k (t)=G_{V"_k} (t − t_k)$, where $t_k$ are the voxel positions in object space
* Object coordinates are transformed to camera coordinates using an affine mapping $\mathbf u = \phi(t)$, called viewing transformation. It is defined by a matrix $\mathbf W$ and a translation vector $\mathbf d$ as $\phi(t) = Wt + d$. We transform the reconstruction kernels $G_{V"k} (t − t_k)$ to camera space by substituting $t = \phi ^{−1}(u)$ and using Equation 2 : 

$G \mathbf{V"_{k}}(\Phi^{-1}(u) - t_k) = \frac{1}{|\mathbf{W}^{-1}|} G_{V'_k } (\mathbf u - \mathbf u_k) = r'_k(u\mathbf).$

where $u_k = \phi(\mathbf t_k)$ is the center of the Gaussian in camera coordinates and $r′_k(\mathbf u)$ denotes the reconstruction kernel in camera space. According to equation 2, the covariance matrix in camera coordinates $V′_k$ is given by $V′_k = WV"_kW^T$ (exactly as detailed in Affine projection of the gauusian).

#### Camera to ray space
Reminder: the local affine transformation - $$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
$\mathbf{m_{u_k}\mathbf(u)}$ transforms from camera to ray, hence $x_k=m(u_k)$ where $x_k$ is the center of the gaussian in ray space. 
since $\mathbf x = m(\mathbf u)$ we get that $ u = \mathbf m^{-1}(\mathbf(x))$ hence: 
$$ u = \mathbf m^{-1}(x) = \mathbf(x - x_k)\cdot \mathbf J^{-1}_{\mathbf u_k} + \mathbf u_k $$
$$ r_k(\mathbf x) = \frac{1}{|\mathbf W^{-1}|}\cdot G_{\mathbf V'_k}(\mathbf{(x - x_k)\cdot J^{-1}_{\mathbf u_k} + \mathbf u_k - \mathbf u_k}) = \\
\frac{1}{|\mathbf W^{-1}||\mathbf J^{-1}|}G_{\mathbf V'_k}(J^{-1}_{\mathbf u_k}\mathbf{(x - x_k)})\\$$

and the new covariance matrix is defined:  $Vk = JV'_{k}$ $J^T = JWV"_kW^TJ^T$.
where V"_k is the original covariance matrix





### Perspective Projection and Jacobian Derivation

In a pinhole camera model, the perspective projection of a 3D point \((X, Y, Z)\) into 2D image coordinates \((x, y)\) can be expressed as:

$$
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix} = \frac{1}{Z} \cdot
\begin{bmatrix}
f_x \cdot X + c_x \cdot Z \\
f_y \cdot Y + c_y \cdot Z \\
Z
\end{bmatrix}
$$

Where:

* $(x, y)$  : are the projected 2D coordinates on the image plane.
* $(X, Y, Z)$  : are the coordinates in the 3D space. 
* $f_x , f_y$  : are the focal lengths in the x and y directions, respectively. 
* $c_x , c_y$  : are the coordinates of the principal point (optical center) in the image. 

### Derivation of the Jacobian

To derive the Jacobian, we compute the partial derivatives of the projected 2D coordinates \((x, y)\) with respect to the 3D coordinates \((X, Y, Z)\).

1. **From the perspective projection equations**:
   - For \(x\):
   $$
   x = \frac{f_x \cdot X + c_x \cdot Z}{Z}
   $$
   - For \(y\):
   $$
   y = \frac{f_y \cdot Y + c_y \cdot Z}{Z}
   $$

2. **Calculating the derivatives**:
   - The derivatives with respect to \(Z\) yield:
     - For \(x\):
     $$
     \frac{\partial x}{\partial X} = \frac{f_x}{Z}, \quad \frac{\partial x}{\partial Y} = 0, \quad \frac{\partial x}{\partial Z} = -\frac{f_x \cdot X}{Z^2}
     $$
     - For \(y\):
     $$
     \frac{\partial y}{\partial X} = 0, \quad \frac{\partial y}{\partial Y} = \frac{f_y}{Z}, \quad \frac{\partial y}{\partial Z} = -\frac{f_y \cdot Y}{Z^2}
     $$

3. **Forming the Jacobian**:
   Using the derivatives calculated above, the Jacobian matrix \(J\) becomes:
   $$
   J = \begin{bmatrix}
   \frac{f_x}{Z} & 0 & -\frac{f_x \cdot X}{Z^2} \\
   0 & \frac{f_y}{Z} & -\frac{f_y \cdot Y}{Z^2}
   \end{bmatrix}
   $$


### 2D Projection 
We calculate the conics coeffitient for each gaussian : $$Ax^2+Bxy+Cy^2 = 0$$, where $A = \frac{\sigma_y^2}{|V|}$, $B = \frac{\sigma_xy}{|V|}$ and  $C = \frac{\sigma_x^2}{|V|}$
we then multiply by the distace between the gaussiam and the pixel. this is the same as: 
$-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})$ - the exponent of the gaussian

wIncorporating the scaled conic, which represents the 2D Gaussian as an ellipse in the Gaussian equation, we can express the effect of every Gaussian on the pixel's color as follows:

$$
G_{\mathbf{V}}(\mathbf{x}) = \frac{1}{(2\pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} ( Ax^2+Bxy+Cy^2)}
$$

This equation captures how each Gaussian contributes to the pixel's color based on its distance from the pixel and the shape defined by the covariance matrix.

## tiles
To render the image we devide it to tiles of $ 16 X 16 $, for each tile we sort the gaussians according to the distance from the camera. we then sample all gaussians effect per pixel in the tile (using the power equation to calculate the opacity). We sum the gaussians color values and multiply by the opacity (doing alpha blending, each time we update the opacity according to the already calculated gaussians $color = RGB * \alpha *\alpha_{blend})$ were for eacg gaussian we add we update $\alpha_{blend} =  \alpha_{blend}*(1-\alpha) $ (when we reach 0, its opaque). to later do the alpha blend we pick for each tile only the gaussians that intersect with the tile. since gaussians are not bounded, we bound them with 3 $\sigma_x$ and 3 $\sigma_y$ bounding box in each axis. 

